In [1]:
import pandas as pd
import timeit
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-py-m.us-central1-b.c.cncd-cncd.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.59-63cf625e29e5
LOGGING: writing to /hail-20201213-0923-0.2.59-63cf625e29e5.log


In [2]:
data = hl.read_matrix_table("gs://pgr-wes-30k-mt/Vep_MT/Promis30_combined.mt")
print("*** Final Merged Data Shape ***")
print("Rows    Columns")
shapeMT = data.count()
print(shapeMT[0], shapeMT[1])

*** Final Merged Data Shape ***
Rows    Columns
5817481 30833


In [3]:
print(data.count())
data1 = hl.split_multi_hts(data)
print(data1.count())

(5817481, 30833)
(6670241, 30833)


In [ ]:
time0 = timeit.default_timer()
pruned_variant_MT = hl.ld_prune(data1.GT, r2=0.2, bp_window_size=75000)
time1 = timeit.default_timer()
print(time1-time0)
filtered_mt =data1.filter_rows(hl.is_defined(pruned_variant_MT[data1.row_key]))
time2 = timeit.default_timer()
print(time2-time1)
filtered_mt.count()

2020-12-13 09:28:47 Hail: INFO: ld_prune: running local pruning stage with max queue size of 8650 variants
2020-12-13 09:38:31 Hail: INFO: wrote table with 5660175 rows in 5540 partitions to /tmp/tSTqBFukNbmijyZwIftHiY
    Total size: 113.59 MiB
    * Rows: 113.59 MiB
    * Globals: 11.00 B
    * Smallest partition: 622 rows (12.02 KiB)
    * Largest partition:  1486 rows (42.40 KiB)
2020-12-13 09:42:53 Hail: INFO: Wrote all 11056 blocks of 5660175 x 30833 matrix with block size 4096.
2020-12-13 09:56:44 Hail: INFO: wrote table with 124031 rows in 2763 partitions to /tmp/BcC4DhQuZObjfLtTBfgpgJ
    Total size: 59.43 MiB
    * Rows: 2.30 MiB
    * Globals: 57.13 MiB
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  552 rows (9.94 KiB)


1706.417287918
0.036596257999917725


(5551173, 30833)

In [8]:
filtered_mt.rows().show()

+---------------+------------+------+----------+
| locus         | alleles    | rsid |     qual |
+---------------+------------+------+----------+
| locus<GRCh37> | array<str> | str  |  float64 |
+---------------+------------+------+----------+
| 1:12198       | ["G","C"]  | NA   | 5.31e+03 |
| 1:12237       | ["G","A"]  | NA   | 8.71e+01 |
| 1:12266       | ["G","A"]  | NA   | 1.38e+03 |
| 1:12559       | ["G","A"]  | NA   | 6.35e+04 |
| 1:12586       | ["C","T"]  | NA   | 1.41e+02 |
| 1:12599       | ["C","A"]  | NA   | 4.32e+04 |
| 1:12608       | ["C","T"]  | NA   | 5.15e+01 |
| 1:12623       | ["A","G"]  | NA   | 5.05e+01 |
| 1:12650       | ["A","T"]  | NA   | 5.34e+01 |
| 1:12659       | ["G","C"]  | NA   | 2.29e+03 |
+---------------+------------+------+----------+

+--------------------------------+--------------+----------------+---------+
| filters                        | info.AC      | info.AF        | info.AN |
+--------------------------------+--------------+----------------+---------+
| set<str>                       | array<int32> | array<float64> |   int32 |
+--------------------------------+--------------+----------------+---------+
| {}                             | [279]        | [4.20e-02]     |    6692 |
| {}                             | [6]          | [3.70e-04]     |   16208 |
| {"VQSRTrancheSNP99.60to99.80"} | [42]         | [2.52e-03]     |   16684 |
| {"VQSRTrancheSNP99.60to99.80"} | [289]        | [5.07e-03]     |   56984 |
| {}                             | [5]          | [8.45e-05]     |   59188 |
| {}                             | [15]         | [2.55e-04]     |   58766 |
| {}                             | [3]          | [4.92e-05]     |   61022 |
| {}                             | [1]          | [1.62e-05]     |   61938 |
| {}                             | [1]          | [1.60e-05]     |   62628 |
| {}                             | [70]         | [1.10e-03]     |   63890 |
+--------------------------------+--------------+----------------+---------+

+-------------------+----------------------+---------+---------+---------+
| info.BaseQRankSum | info.ClippingRankSum | info.DB | info.DP | info.DS |
+-------------------+----------------------+---------+---------+---------+
|           float64 |              float64 |    bool |   int32 |    bool |
+-------------------+----------------------+---------+---------+---------+
|          0.00e+00 |             0.00e+00 |   false |    6739 |   false |
|          7.36e-01 |            -3.58e-01 |   false |   11753 |   false |
|          1.03e+00 |             0.00e+00 |   false |   13364 |   false |
|          1.23e+00 |             0.00e+00 |   false |  117509 |   false |
|          1.73e+00 |            -1.24e-01 |   false |  129308 |   false |
|          1.03e+00 |             1.98e-01 |   false |  120093 |   false |
|          7.31e-01 |            -7.31e-01 |   false |  143332 |   false |
|          1.98e-01 |             9.22e-01 |   false |  156225 |   false |
|          1.98e-01 |            -1.98e-01 |   false |  153502 |   false |
|         -3.58e-01 |            -3.58e-01 |   false |  179304 |   false |
+-------------------+----------------------+---------+---------+---------+

+----------+----------+---------------------+----------------------+
| info.END |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+----------+----------+---------------------+----------------------+
|    int32 |  float64 |             float64 |              float64 |
+----------+----------+---------------------+----------------------+
|       NA | 0.00e+00 |                  NA |            -2.20e-02 |
|       NA | 0.00e+00 |                  NA |            -1.52e-01 |
|       NA | 0.00e+00 |                  NA |            -1.46e-01 |
|       NA | 0.00e+00 |                  NA |            -7.33e-02 |
|       NA | 0.00e+00 |                  NA |            -6.28e-02 |
|       NA | 0.00e+00 |                  NA |            -6.72e-02 |
|       NA | 0.00e+00 

In [ ]:
# HWE_Normalized_PCA
eigenvalues, scores, loadings = hl.hwe_normalized_pca(filtered_mt.GT, k=10)

2020-12-13 10:02:16 Hail: INFO: hwe_normalized_pca: running PCA using 5551173 variants.
2020-12-13 10:05:10 Hail: INFO: pca: running PCA with 10 components...


In [ ]:
# PCA
eigenvalues1, scores1, loadings1 = hl.pca(hl.int(hl.is_defined(filtered_mt.GT)), k=10)

2020-12-13 12:00:21 Hail: INFO: pca: running PCA with 10 components...


In [ ]:
mt = filtered_mt.annotate_cols(pca_hwe_scores = scores[filtered_mt.s].scores)

In [ ]:
mt1 = mt.annotate_cols(pca_scores = scores1[mt.s].scores)

In [ ]:
mt1.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'pca_hwe_scores': array<float64>
    'pca_scores': array<float64>
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSITIVE_TRAIN_SITE: bool, 
        QD: float64, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        VQSLOD: float64, 
      

In [ ]:
mt1.cols().show()

2020-12-13 12:22:43 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2020-12-13 12:22:45 Hail: INFO: Coerced sorted dataset


,,
s,pca_hwe_scores,pca_scores
str,array<float64>,array<float64>
"""AY1002""","[-2.58e-04,-1.57e-03,-2.75e-04,1.54e-04,1.73e-04,1.03e-02,-3.52e-04,9.83e-04,-7.85e-05,-1.91e-04]","[-2.34e+03,5.33e+01,5.73e+01,4.04e+00,5.90e+00,1.10e+01,1.39e+01,-3.44e+00,-6.53e+00,-1.40e+00]"
"""AY1012""","[-2.00e-04,-1.99e-03,-4.83e-04,3.32e-04,3.99e-04,1.07e-02,-3.64e-04,1.05e-03,-2.63e-04,-1.01e-04]","[-2.34e+03,5.38e+01,5.64e+01,1.29e+00,3.86e+00,1.16e+01,1.66e+01,-3.29e+00,-9.67e-01,-4.61e+00]"
"""AY1021""","[-2.31e-04,-2.15e-03,-3.73e-04,1.91e-04,4.05e-04,9.76e-03,-3.84e-04,9.14e-04,-2.01e-04,-8.71e-05]","[-2.34e+03,3.86e+01,5.31e+01,-7.82e+01,-7.97e+00,1.42e+01,1.13e+01,4.87e+00,2.84e+00,-1.48e+01]"
"""AY1022""","[-2.23e-04,-2.06e-03,-3.44e-04,4.10e-04,3.77e-04,1.17e-02,-3.67e-04,1.33e-03,-3.11e-04,-7.74e-05]","[-2.34e+03,5.52e+01,6.56e+01,6.90e+00,5.50e+00,1.07e+01,8.28e+00,-3.36e+00,-1.66e+01,8.11e+00]"
"""AY1024""","[-2.10e-04,-1.94e-03,-4.03e-04,3.18e-04,2.58e-04,9.98e-03,-4.28e-04,1.12e-03,-1.61e-04,-5.82e-05]","[-2.34e+03,5.15e+01,6.25e+01,-1.56e+01,4.99e-01,-4.20e+01,2.08e+01,-4.46e+00,-1.04e+01,-7.63e+00]"
"""AY1026""","[-2.86e-04,-1.79e-03,-3.20e-04,2.24e-04,2.14e-04,1.08e-02,-4.68e-04,1.06e-03,-1.42e-04,-1.61e-04]","[-2.34e+03,5.50e+01,5.30e+01,2.88e+01,5.82e+00,8.68e+00,-4.85e+00,-5.95e+00,-6.09e+00,1.20e+01]"
"""AY1031""","[-2.59e-04,-2.44e-03,-7.41e-04,4.51e-04,5.43e-04,1.05e-02,-3.05e-04,1.70e-03,-8.00e-04,2.59e-04]","[-2.34e+03,5.43e+01,5.57e+01,2.07e+01,2.62e+00,-4.49e+01,2.96e+00,-8.29e+00,-1.99e+00,3.80e+00]"
"""AY1034""","[-1.89e-04,-2.41e-03,-6.83e-04,4.68e-04,5.16e-04,1.21e-02,-4.36e-04,1.31e-03,-4.35e-04,-1.86e-05]","[-2.34e+03,5.56e+01,5.68e+01,2.61e+01,6.76e+00,9.72e+00,-4.57e+00,-4.31e+00,-1.05e+01,1.77e+01]"


In [ ]:
mt1.write("gs://pgr-wes-30k-mt/Vep_MT/30K_LD-Pruned_PCA.mt")

2020-12-13 12:35:30 Hail: INFO: wrote matrix table with 5551173 rows and 30833 columns in 5540 partitions to gs://pgr-wes-30k-mt/Vep_MT/30K_LD-Pruned_PCA.mt
    Total size: 609.71 GiB
    * Rows/entries: 609.71 GiB
    * Columns: 4.89 MiB
    * Globals: 11.00 B
    * Smallest partition: 364 rows (51.44 MiB)
    * Largest partition:  1360 rows (143.22 MiB)


In [19]:
mt1.cols().export("gs://pgr-wes-30k-mt/Vep_MT/30k_LD-Pruned_PCA.tsv")

2020-12-13 13:55:54 Hail: INFO: Coerced sorted dataset
2020-12-13 13:55:55 Hail: INFO: merging 16 files totalling 12.4M...
2020-12-13 13:55:56 Hail: INFO: while writing:
    gs://pgr-wes-30k-mt/Vep_MT/30k_LD-Pruned_PCA.tsv
  merge time: 706.946ms


In [18]:
!ls

bin					    mnt
boot					    opt
dev					    proc
etc					    root
hadoop					    run
hail-20201213-0855-0.2.59-63cf625e29e5.log  sbin
hail-20201213-0923-0.2.59-63cf625e29e5.log  sparkmonitor_kernelextension.log
home					    srv
initrd.img				    sys
initrd.img.old				    tmp
lib					    usr
lib64					    var
lost+found				    vmlinuz
media					    vmlinuz.old


In [ ]:
R2 = [0.05,0.1,0.5,0.9]
win_size = list(range(20000,510000,20000))

parent_df = pd.DataFrame()
for r2 in R2:
    for ws in win_size:
        print("R2-",r2,"    Window Size-",ws)
        start = timeit.default_timer()
        data = hl.read_matrix_table("gs://pgr-wes-30k-mt/Complete_Annotations/Canonical/30K_Chr21_VEP_DB_C.mt")
        data = hl.split_multi_hts(data)
        data_ld = hl.ld_prune(data.GT,r2=r2, bp_window_size=ws)
        stop = timeit.default_timer()
        a=data.count()[0]
        time=stop - start
        parent_df = parent_df.append([['Chr1',r2,ws,data_ld.count(),a,str(round((time/60),2))+" Min"]])
        print(round((time/60),2), "Min")
print("Exporting Excel To GCS")
parent_df.to_excel(str(r2)+"LD-Prune_CHR21.xlsx")
!gsutil cp *.xlsx gs://pgr-wes-30k-mt/Ld_Pruning_Results/
print("Done")

In [ ]:
parent_df.to_excel("Chr1_30k_LdPruning_Results.xlsx")
!gsutil cp *.xlsx gs://pgr-wes-30k-mt/Ld_Pruning_Results/

In [ ]:
data_ld.globals().show()

In [ ]:
data.count()[0]

In [ ]:


pd.DataFrame(data_ld_collect).to_excel('LD_Pruned.xlsx', header=False, index=False)
!gsutil cp LD_Pruned.xlsx gs://pgr-wes-30k-mt/